In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/input/combined.csv')
df.info()

concepts = pd.read_csv('../data/input/concepts.csv')
concepts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76893 entries, 0 to 76892
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   parameter_name  76893 non-null  object 
 1   concept_label   76893 non-null  object 
 2   hospital_name   76893 non-null  object 
 3   ehr_name        76893 non-null  object 
 4   table           76893 non-null  object 
 5   parameter_id    55008 non-null  object 
 6   atc             8840 non-null   object 
 7   unit            76893 non-null  object 
 8   num_units       75621 non-null  float64
 9   amin            41851 non-null  float64
 10  amax            41851 non-null  float64
 11  p25             41851 non-null  float64
 12  p50             41851 non-null  float64
 13  p75             41851 non-null  float64
 14  top_n           76893 non-null  object 
 15  num_records     76892 non-null  float64
 16  num_patients    75621 non-null  float64
 17  perc_patients   75502 non-null 

In [8]:
r_parameters = df.parameter_name.shape[0]
r_parameters_unique = df.parameter_name.nunique()
r_parameters_unique_proportion = r_parameters_unique / r_parameters
r_mapped = (df.concept_label != 'unmapped').sum()
r_mapped_proportion = r_mapped / r_parameters
r_mapped_categories = df.concept_label.map(concepts.set_index('concept_label')['category'])
r_duplicates = df.loc[df.duplicated(subset=['parameter_name', 'hospital_name', 'table', 'parameter_id'], keep=False)].shape[0]
r_mapped_number = df.loc[df.concept_label != 'unmapped']['concept_label'].nunique()
r_mapped_non_med = df.loc[(df.concept_label != 'unmapped') & (~df.concept_label.str.startswith('med_'))]['concept_label'].nunique()

In [4]:
r_groups = concepts.loc[concepts['category'] != 'medication'].groupby('category').count().index.nunique()
r_labels = concepts.loc[concepts['category'] != 'medication'].groupby('category').count().sum()['concept_label']
r_labels_2 = concepts.loc[concepts['category'] != 'medication'].groupby('category', dropna=False).count().sum()['concept_label']
r_medication = concepts.groupby('category').count()['concept_label']['medication']

In [10]:
print(f"""
The parameter file contains {r_parameters} parameters, of which {r_parameters_unique} are unique ({r_parameters_unique_proportion:.2%}). Based on parameter name, hospital name, table of origin and parameter id, {r_duplicates} records are marked as full duplicates.

 Of all parameters, {r_mapped} are mapped to a concept label. This is {r_mapped_proportion:.2%} of the total parameters. The mapped parameters are distributed as follows:

{r_mapped_categories.value_counts(normalize=True).head(10).apply(lambda x: f'{x:.2%}')}

In total, {r_mapped_number} unique concepts were used to map the parameters. Of these, {r_mapped_non_med} are not medication concepts and {r_mapped_number - r_mapped_non_med} medication records.

There are {r_groups} groups of concepts and {r_labels} ({r_labels_2}) labels in the concepts file. With an additional {r_medication} medication concepts in the combined file.
""")


The parameter file contains 76893 parameters, of which 58848 are unique (76.53%). Based on parameter name, hospital name, table of origin and parameter id, 0 records are marked as full duplicates.

 Of all parameters, 38824 are mapped to a concept label. This is 50.49% of the total parameters. The mapped parameters are distributed as follows:

unmapped                     49.51%
medication                   15.28%
laboratory value             11.88%
respiratory                   8.49%
fluid balance                 5.23%
hemodynamics                  4.49%
neurology                     1.02%
infectiology                  0.94%
clinical score                0.82%
renal replacement therapy     0.62%
Name: concept_label, dtype: object

In total, 1679 unique concepts were used to map the parameters. Of these, 837 are not medication concepts and 842 medication records.

There are 15 groups of concepts and 982 (982) labels in the concepts file. With an additional 5456 medication concepts in 

In [6]:
concepts.head()

,concept_label,concept_label_super,category
0,access_pressure_cvvh,NaN,renal replacement therapy
1,access_pressure_cvvhd,NaN,renal replacement therapy
2,aciclovir_level,NaN,laboratory value
3,aciclovir_trough_level,NaN,laboratory value
4,activated_clotting_time,NaN,laboratory value
